
# カタカナから漢字・アルファベットを推測するモデルを学習する
オープンソース辞書及び，Wikipediaから作成した漢字とカタカナのペアデータを用いて，Transformer+Seq2Seqのモデルで，スクラッチで学習する。

出力の文字（漢字・アルファベットなど）は一文字ずつに分割，また，入力の文字（カタカナ）も一文字ずつに分割して，学習する。



## データ準備
データは，[dict](../dict)で作成した姓名辞書データのoss.json及びseimei.jsonと，Wikipediaから抜き出した，漢字姓名とカタカナ姓名（スペース区切りのもの）を用いて，学習に使用した。単語単位である姓名辞書データと，Wikipediaのスペースで区切られた姓名を用いることで，姓名で漢字とカナが入力された場合の学習を可能とした。

また，ひらがなとカタカナ，


In [1]:
# スクリプトで使う環境変数をセット
import os
os.environ['DIR']='dataset_r'
os.environ['MODEL']='model_r'
os.environ["LOG"]='logs_r'
os.environ['RATIO']="0.01"
os.environ['WIKI']="../dict/wikipedia/wikiname.txt"

os.makedirs(os.environ['DIR'],exist_ok=True)
os.makedirs(os.environ['MODEL'],exist_ok=True)

In [2]:
# オープンソース辞書とWikipediaから作成した単語辞書をマージする
!python mergedic.py --dic1 ../dict/oss.json --dic2 ../dict/seimei.json --outfile tmp.json



{
  "dic1": "../dict/oss.json",
  "dic2": "../dict/seimei.json",
  "outfile": "tmp.json"
}
len=2150277
cnt=2252683


In [3]:
# 作成したデータを訓練用，開発用，検証用に分ける。開発用と検証用がそれぞれ全体の0.01
!python prep.py --json tmp.json --outdir $DIR --ratio $RATIO --reverse 


{
  "json": "tmp.json",
  "outdir": "dataset_r",
  "ratio": 0.01,
  "reverse": true
}
k_max_len=72
v_max_len=120
train,2207631
valid,22526
test,22526


In [4]:
# wikipediaの漢字姓名とカナ姓名を取得し，訓練，開発，検証用に分ける。
!python catawk.py --begin_idx_per 0 --end_idx_per 1 --appendfile $DIR/valid.src --index 1 --infile $WIKI --infile_delimiter tsv
!python catawk.py --begin_idx_per 0 --end_idx_per 1 --appendfile $DIR/valid.tgt --index 0 --infile $WIKI --infile_delimiter tsv
!python catawk.py --begin_idx_per 1 --end_idx_per 2 --appendfile $DIR/test.src --index 1 --infile $WIKI --infile_delimiter tsv
!python catawk.py --begin_idx_per 1 --end_idx_per 2 --appendfile $DIR/test.tgt --index 0 --infile $WIKI --infile_delimiter tsv
!python catawk.py --begin_idx_per 2 --end_idx_per 100 --appendfile $DIR/train.src --index 1 --infile $WIKI --infile_delimiter tsv
!python catawk.py --begin_idx_per 2 --end_idx_per 100 --appendfile $DIR/train.tgt --index 0 --infile $WIKI --infile_delimiter tsv

In [5]:
# ひらがなとカタカナのペアデータを作成し，訓練データに混ぜる
!python kana.py --outfile $DIR/hirakata.txt --reverse

!python catawk.py --begin_idx_per 0 --end_idx_per 100 --appendfile $DIR/train.src --index 0 --infile $DIR/hirakata.txt
!python catawk.py --begin_idx_per 0 --end_idx_per 100 --appendfile $DIR/train.tgt --index 1 --infile $DIR/hirakata.txt


{
  "outfile": "dataset_r/hirakata.txt",
  "reverse": true
}


In [6]:
# 単漢字とカタカナのデータを，訓練データに混ぜる 
!python jsontotext.py --jsonfile ../dict/tankanji.json --outfile $DIR/tankanji.txt --reverse

!python catawk.py --begin_idx_per 0 --end_idx_per 100 --appendfile $DIR/train.src --index 0 --infile $DIR/tankanji.txt
!python catawk.py --begin_idx_per 0 --end_idx_per 100 --appendfile $DIR/train.tgt --index 1 --infile $DIR/tankanji.txt

{
  "jsonfile": "../dict/tankanji.json",
  "outfile": "dataset_r/tankanji.txt",
  "reverse": true
}


In [7]:
# 開発，検証用に，訓練データに入っているものがあれば，削除する(インサンプルを阻止)
!python omit_testval.py --train_src $DIR/train.src --train_tgt $DIR/train.tgt --test_src $DIR/test.src --test_tgt $DIR/test.tgt --valid_src $DIR/valid.src --valid_tgt $DIR/valid.tgt



{
  "train_src": "dataset_r/train.src",
  "train_tgt": "dataset_r/train.tgt",
  "test_src": "dataset_r/test.src",
  "test_tgt": "dataset_r/test.tgt",
  "valid_src": "dataset_r/valid.src",
  "valid_tgt": "dataset_r/valid.tgt"
}
train=2492861
omit=533


In [8]:
# 訓練，開発，検証データをシャッフルする

!python shuffle.py --src $DIR/train.src --tgt $DIR/train.tgt
!python shuffle.py --src $DIR/valid.src --tgt $DIR/valid.tgt
!python shuffle.py --src $DIR/test.src --tgt $DIR/test.tgt



{
  "tgt": "dataset_r/train.tgt",
  "src": "dataset_r/train.src"
}
{
  "tgt": "dataset_r/valid.tgt",
  "src": "dataset_r/valid.src"
}
{
  "tgt": "dataset_r/test.tgt",
  "src": "dataset_r/test.src"
}


In [9]:
# データをスペース区切りにする（文字単位で，学習させるため）

!python space.py --infile $DIR/train.src --outfile $DIR/train.src
!python space.py --infile $DIR/train.tgt --outfile $DIR/train.tgt
!python space.py --infile $DIR/valid.src --outfile $DIR/valid.src
!python space.py --infile $DIR/valid.tgt --outfile $DIR/valid.tgt
!python space.py --infile $DIR/test.src --outfile $DIR/test.src
!python space.py --infile $DIR/test.tgt --outfile $DIR/test.tgt



{
  "infile": "dataset_r/train.src",
  "outfile": "dataset_r/train.src"
}
{
  "infile": "dataset_r/train.tgt",
  "outfile": "dataset_r/train.tgt"
}
{
  "infile": "dataset_r/valid.src",
  "outfile": "dataset_r/valid.src"
}
{
  "infile": "dataset_r/valid.tgt",
  "outfile": "dataset_r/valid.tgt"
}
{
  "infile": "dataset_r/test.src",
  "outfile": "dataset_r/test.src"
}
{
  "infile": "dataset_r/test.tgt",
  "outfile": "dataset_r/test.tgt"
}


In [10]:
# jsonl形式に変換する

!python format.py --src $DIR/train.src --tgt $DIR/train.tgt --outfile $DIR/train.jsonl --src_key kana --tgt_key kanji
!python format.py --src $DIR/test.src --tgt $DIR/test.tgt --outfile $DIR/test.jsonl --src_key kana --tgt_key kanji
!python format.py --src $DIR/valid.src --tgt $DIR/valid.tgt --outfile $DIR/valid.jsonl --src_key kana --tgt_key kanji

## カタカナと漢字・アルファベットとのペアから学習する
学習は，訓練用データを用いて学習し，１エポックごとに開発用データ(valid.json)でLossを確認する。


### パラメタ
- emb_size 
入力，出力の文字のエンベッドのDimension

- nhead
マルチヘッド数

- ffn_hid_dim
FFNのDimension

- num_encoder_layer
エンコードレイヤーの数

- num_decoder_layer
デコードレイヤーの数

- lr 
学習率

- dropout
ドロップアウトの割合, 0-1

- num_epochs
何周学習データを用いて学習を行うか

- device
mps,cpu,cudaから選択
-- mps
アップルシリコンを搭載したマシンで実行する際に選択
-- cuda
CUDAが利用できる環境で選択
-- cpu
上記以外は，CPUモードを選択

- earlystop_patient
開発用データでlossが下がらなくなってearlystop_patient回計算が終了した場合に，num_epochs数以下でも，計算を終了させる

- output_dir
モデルの出力ディレクトリ

- tensorboard_logdir
tensorboard形式のログの出力ディレクトリ。学習状況を確認するためには`tensorboard --logdir logs`を実行後，ブラウザでhttp://localhost:6000/から確認
 



- prefix
jsonl形式のデータのprefix

- source_lang
jsonl形式のデータのsourceのキー

- target_lang
jsonl形式のデータのtargetのキー

- train_file
訓練用データのJSONLファイル

- valid_file
開発用データのJSONLファイル

In [11]:
# 訓練実行

!python ./transformer_model.py \
  --emb_size 512 \
  --nhead 8 \
  --ffn_hid_dim 2048 \
  --batch_size 32 \
  --num_encoder_layers 8 \
  --num_decoder_layers 8 \
  --lr 0.00002 \
  --dropout 0.3 \
  --num_epochs 50 \
  --device cuda \
  --earlystop_patient 3 \
  --output_dir $MODEL \
  --tensorboard_logdir $LOG \
  --prefix translation \
  --source_lang kana \
  --target_lang kanji \
  --train_file $DIR/train.jsonl \
  --valid_file $DIR/valid.jsonl


num_epochs:50
Epoch: 1, Train loss: 4.625, Val loss: 3.484, Epoch time = 10628.873s
Epoch: 2, Train loss: 3.281, Val loss: 2.538, Epoch time = 10621.585s
Epoch: 3, Train loss: 2.573, Val loss: 1.879, Epoch time = 10606.072s
Epoch: 4, Train loss: 2.089, Val loss: 1.494, Epoch time = 10591.852s
Epoch: 5, Train loss: 1.780, Val loss: 1.274, Epoch time = 10599.757s
Epoch: 6, Train loss: 1.578, Val loss: 1.129, Epoch time = 10607.291s
Epoch: 7, Train loss: 1.439, Val loss: 1.031, Epoch time = 10615.778s
Epoch: 8, Train loss: 1.337, Val loss: 0.962, Epoch time = 10622.484s
Epoch: 9, Train loss: 1.261, Val loss: 0.913, Epoch time = 10615.375s
Epoch: 10, Train loss: 1.200, Val loss: 0.871, Epoch time = 10616.288s
Epoch: 11, Train loss: 1.152, Val loss: 0.841, Epoch time = 10607.549s
Epoch: 12, Train loss: 1.112, Val loss: 0.812, Epoch time = 10600.979s
Epoch: 13, Train loss: 1.078, Val loss: 0.788, Epoch time = 10604.699s
Epoch: 14, Train loss: 1.048, Val loss: 0.769, Epoch time = 10601.055s
E

In [19]:
%load_ext tensorboard


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [20]:
%tensorboard --logdir logs_r

In [21]:
#追加学習

!python ./transformer_model.py \
  --emb_size 512 \
  --nhead 8 \
  --ffn_hid_dim 2048 \
  --batch_size 32 \
  --num_encoder_layers 8 \
  --num_decoder_layers 8 \
  --lr 0.00002 \
  --dropout 0.3 \
  --num_epochs 100 \
  --device cuda \
  --earlystop_patient 3 \
  --output_dir $MODEL \
  --tensorboard_logdir $LOG \
  --prefix translation \
  --source_lang kana \
  --target_lang kanji \
  --train_file $DIR/train.jsonl \
  --valid_file $DIR/valid.jsonl

load:model_r/checkpoint_050.pt,now_epoch=50,best_loss=None
num_epochs:100
Epoch: 51, Train loss: 0.651, Val loss: 0.531, Epoch time = 10630.201s
Epoch: 52, Train loss: 0.648, Val loss: 0.529, Epoch time = 10625.721s
Epoch: 53, Train loss: 0.645, Val loss: 0.528, Epoch time = 10608.591s
Epoch: 54, Train loss: 0.642, Val loss: 0.527, Epoch time = 10606.666s
Epoch: 55, Train loss: 0.639, Val loss: 0.527, Epoch time = 10605.226s
Epoch: 56, Train loss: 0.636, Val loss: 0.525, Epoch time = 10591.121s
Epoch: 57, Train loss: 0.633, Val loss: 0.525, Epoch time = 10588.156s
Epoch: 58, Train loss: 0.631, Val loss: 0.522, Epoch time = 10596.907s
Epoch: 59, Train loss: 0.628, Val loss: 0.522, Epoch time = 10598.710s
Epoch: 60, Train loss: 0.626, Val loss: 0.522, Epoch time = 10602.947s
Epoch: 61, Train loss: 0.623, Val loss: 0.521, Epoch time = 10597.646s
Epoch: 62, Train loss: 0.621, Val loss: 0.519, Epoch time = 10590.451s
Epoch: 63, Train loss: 0.619, Val loss: 0.517, Epoch time = 10592.195s
Epo

## モデルを検証する
作成したモデルを，検証用データを用いて検証し，正解率を算出する


## パラメタ

- test_file    
jsonl形式の検証用データファイル

- model_file    
transformer_model.pyで作成したモデルファイル。

- outfile    
検証結果を格納するファイル

- device    
cpu限定

- nbest    
検証用データの漢字姓名を入力データとした時に，モデルが出力するカタカナ姓名を確率の高い方からnbest個出力する。beam_width以下。searchパラメタがbothかbeamの時有効

- beam_width    
ビームサーチのビーム幅。searchパラメタがbothかbeamの時有効

- max_len    
出力するカタカナ姓名の最大長さ

- search    
greedy,beam,bothから選択    
  - greedy    
貪欲法による探索    
  - beam    
ビームサーチによる探索    
  - both    
貪欲砲，ビームサーチ両方の探索    



In [12]:
# 検証用データを実行


!python generate_batch.py \
  --test_file $DIR/test.jsonl \
  --model_file $MODEL/checkpoint_best.pt \
  --outfile $DIR/generate.txt \
  --device cpu \
  --nbest 5 \
  --beam_width 5 \
  --max_len 100 \
  --search both





Traceback (most recent call last):
  File "/home/analysis01/src/kanjikana-model/train/generate_batch.py", line 18, in <module>
    from training.model import KanjiKanaTransformer, KanjiKanaDataSet, EOS_IDX, BOS_IDX, SPECIAL_SYMBOLS, engfra_tokenizer
ModuleNotFoundError: No module named 'training'


In [13]:
# 検証結果を評価
!python evaluate.py --infile $DIR/generate.txt



Traceback (most recent call last):
  File "/home/analysis01/src/kanjikana-model/train/evaluate.py", line 54, in <module>
    main()
  File "/home/analysis01/src/kanjikana-model/train/evaluate.py", line 51, in main
    run(args)
  File "/home/analysis01/src/kanjikana-model/train/evaluate.py", line 15, in run
    with open(args.infile,'r',encoding='utf-8') as f:
FileNotFoundError: [Errno 2] No such file or directory: 'dataset_r/generate.txt'


# モデルをJavaのDJL用に変換する
Pytorchで学習したモデルをJavaで使えるようにモデルの変換を行う。



In [14]:
# カナから漢字
!python convert_jitscript.py  \
    --model_file=$MODEL_R/checkpoint_best.pt \
    --model_script=$MODEL_R/script.pt \
    --encoder=$MODEL_R/encoder.pt \
    --decoder=$MODEL_R/decoder.pt \
    --positional_encoding=$MODEL_R/positional_encoding.pt \
    --generator=$MODEL_R/generator.pt \
    --src_tok_emb=$MODEL_R/src_tok_emb.pt \
    --tgt_tok_emb=$MODEL_R/tgt_tok_emb.pt \
    --vocab_src=$MODEL_R/vocab_src.txt \
    --vocab_tgt=$MODEL_R/vocab_tgt.txt \
    --params=$MODEL_R/params.json \
    --device=cpu



Traceback (most recent call last):
  File "/home/analysis01/src/kanjikana-model/train/convert_jitscript.py", line 79, in <module>
    main()
  File "/home/analysis01/src/kanjikana-model/train/convert_jitscript.py", line 76, in main
    KanjiKanaTransformerScripted(args).convert()
  File "/home/analysis01/src/kanjikana-model/train/convert_jitscript.py", line 26, in convert
    best_checkpoint = torch.load(self.args.model_file, map_location=torch.device(self.args.device))
  File "/home/analysis01/src/kanjikana-model/venv/lib/python3.10/site-packages/torch/serialization.py", line 1319, in load
    with _open_file_like(f, "rb") as opened_file:
  File "/home/analysis01/src/kanjikana-model/venv/lib/python3.10/site-packages/torch/serialization.py", line 659, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/home/analysis01/src/kanjikana-model/venv/lib/python3.10/site-packages/torch/serialization.py", line 640, in __init__
    super().__init__(open(name, mode))
FileNotFou